In [4]:
import pandas as pd
import os

# === 1. CSV 파일 경로 설정 ===
csv_path = "../data_csv/11_09/강원도_사업체_공간집계정보.csv"    # 네가 가진 CSV 파일 이름 그대로 입력
output_sql_path = "insert_kangwondo_Spatial_statistics.sql"

# === 2. CSV 불러오기 (UTF-8 → CP949 순차 시도) ===
try:
    df = pd.read_csv(csv_path, encoding="utf-8")
except:
    df = pd.read_csv(csv_path, encoding="cp949")

# === 3. SQL 테이블 이름 ===
table_name = "gangwon_market"

# === 4. pandas dtype → SQL 타입 매핑 ===
dtype_map = {
    "object": "VARCHAR(255)",
    "int64": "BIGINT",
    "float64": "DOUBLE"
}

# === 5. CREATE TABLE 생성 ===
sql_lines = []
sql_lines.append(f"DROP TABLE IF EXISTS {table_name};\n")
sql_lines.append(f"CREATE TABLE {table_name} (")

col_defs = []
for col, dtype in df.dtypes.items():
    sql_type = dtype_map.get(str(dtype), "VARCHAR(255)")
    col_defs.append(f"    `{col}` {sql_type}")

sql_lines.append(",\n".join(col_defs))
sql_lines.append(");\n")

# === 6. INSERT INTO 생성 ===
for _, row in df.iterrows():
    values = []
    for v in row:
        if pd.isna(v):
            values.append("NULL")
        elif isinstance(v, (int, float)):
            values.append(str(v))
        else:
            v = str(v).replace("'", "\\'")
            values.append(f"'{v}'")
    sql_lines.append(f"INSERT INTO {table_name} VALUES ({', '.join(values)});")

# === 7. SQL 파일로 저장 ===
with open(output_sql_path, "w", encoding="utf-8") as f:
    f.write("\n".join(sql_lines))

print("SQL 파일 생성 완료:", output_sql_path)

SQL 파일 생성 완료: insert_kangwondo_Spatial_statistics.sql
